In [9]:

import numpy as np
import pandas as pd
import time
import seaborn as sns
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
import itertools
from copy import deepcopy as dcopy,copy
from skimage.morphology import label

In [10]:
# Working through the hooks puzzles.
# Different to #2 as many more limitations on potential hook placements. 24 instead of 3200
# need to work out a good method of solving !
# Brute force backtracking works OK for the outside rows. May need to solve top down and bottom up then check he middle row.
# the connected condition is very important on the inner squares.

### Puzzle details
<img src="https://www.janestreet.com/puzzles/wp-content/uploads/2018/02/hook3_puzzle-01.png" width="300" height="300">

In [11]:
#Setup the constraints
top_labels = [0,35,42,18,18,0, 36, 63,0]
bot_labels = [0,40,32,40,10,12,0,56,0]
left_labels = [0,56,0,32,40,15,16,25,0]
right_labels =[0,49,63,0,18,42,63,54,0]
params = [top_labels,bot_labels,left_labels,right_labels]

In [182]:
class Matrix():
    def __init__(self,top_labels,bot_labels,left_labels,right_labels):
        self.top_labels = top_labels
        self.bot_labels = bot_labels
        self.left_labels = left_labels
        self.right_labels = right_labels
        self.potential_grids = [[-9,np.ones((9,9),dtype=int)*-1,[0,0,9,9]]] # list of all grids not yet ruled out.[Level,grid,coordinates of the grid yet to be filled]
        self.solution = []
        self.splits =  [list(i) for i in itertools.product([0, 1], repeat=9) if sum(i) > 1]
     
    def add_layer(self,grid,coords,lvl,alignment):
        row_start,col_start,row_end, col_end = coords
    
        if alignment == 0:
            grid[row_start:row_end,col_start] =lvl
            grid[row_start,col_start:col_end] =lvl
            row_start +=1
            col_start +=1
    
        if alignment == 1:
            grid[row_start:row_end,col_start] =lvl
            grid[row_end-1,col_start:col_end] =lvl
            row_end -=1
            col_start +=1 

        if alignment == 2:
            grid[row_start:row_end,col_end-1] =lvl
            grid[row_start,col_start:col_end] =lvl
            row_start +=1
            col_end -=1
        
        if alignment == 3:
            grid[row_start:row_end,col_end-1] =lvl
            grid[row_end-1,col_start:col_end] =lvl
            row_end -=1
            col_end -=1 
    
        coords = [row_start,col_start,row_end, col_end]
        
        return grid,coords    
   
    def check_grid(self,grid):
        isValid = 1
        for i in range(9):
            row = grid[i,:]
            col = grid[:,i]
            if -1 not in row:
                    isValid *= self.check_line(row,self.left_labels[i],self.right_labels[i])
            if -1 not in col:
                    isValid *= self.check_line(col,self.top_labels[i],self.bot_labels[i])  
        return isValid
                    
    def check_line(self,line,start,end):
        for split in self.splits:
            test = line * split
            if (test[test!=0][:2].prod() == start) or (start ==0):
                if (test[test!=0][-2:].prod() == end) or (end ==0):
                    return 1
        return 0
    
    


    
    def column_check(self,row,split,solution,y):
        temp = dcopy(solution)
        temp[row,:] = split
        
        #check connectivuty
        if np.max(label(temp!=0,connectivity=1)) > 1 :
            return False
        
        temp *= y
        for i in range(9):
            col = temp[:,i]
            if np.sum(col > 0) == 1:
                if self.top_labels[i] % col[col>0]  != 0:
                    return False
            elif np.sum(col>0) > 1:
                if col[col!=0][:2].prod() != self.top_labels[i]:
                    return False
            if row ==8:
                 if col[col!=0][-2:].prod() != self.bot_labels[i]:
                    return False
        return True   
    
    
    def solve(self):
        
        correct = np.array([[9, 9, 9, 9, 9, 9, 9, 9, 9],
                    [8, 7, 7, 7, 7, 7, 7, 7, 9],
                    [8, 6, 6, 6, 6, 6, 6, 7, 9],
                    [8, 5, 4, 3, 2, 1, 6, 7, 9],
                    [8, 5, 4, 3, 2, 2, 6, 7, 9],
                    [8, 5, 4, 3, 3, 3, 6, 7, 9],
                    [8, 5, 4, 4, 4, 4, 6, 7, 9],
                    [8, 5, 5, 5, 5, 5, 6, 7, 9],                          
                    [8, 8, 8, 8, 8, 8, 8, 8, 9]])
            
        while len(self.potential_grids) > 0:
            
            temp_grid = self.potential_grids.pop(0)
            #create the potential rotations at the given level
            rotations = []
        
            for alignment in range(4):
                lvl,grid,coords = dcopy(temp_grid)
                grid,coords = self.add_layer(grid,coords,-lvl,alignment)
                if lvl != -1 :
                    rotations.append([lvl+1,grid,coords])
                else:
                     rotations = [[lvl+1,grid,coords]]
    
            
            #check valid grids (where the sum can be made from available digits) and save the ones that work
            for i in range(len(rotations)):
                lvl,g,coords = rotations[i]
                if self.check_grid(g):
                    if lvl !=0:    
                        self.potential_grids.append([lvl,g,coords])
                    else:
                        self.solution.append(g)
                        
        #self.solution = []
        #self.solution = [correct]
                        
        print("There are {} valid hooks".format(len(self.solution)))
        print("Check correct grid:",any(np.array_equal(correct, i) for i in self.solution))
        
        #solve each grid in the cut down list
        for i in range(len(self.solution)):
            grid = self.solution[i]
            #self.solve_matrix(grid)
            
                            
def sol_print(solved,matrix):
    fig,ax = plt.subplots(1,1,figsize=(5,5))
    ax = sns.heatmap(matrix,annot=solved*matrix,cbar=False,cmap="YlGnBu")
    ax.axis("off")
    
    

In [183]:
start = time.perf_counter()
test = Matrix(top_labels,bot_labels,left_labels,right_labels)
test.solve()
solved,matrix = test.final
stop =  time.perf_counter()
print('Solution took {:0.4f} seconds\n'.format((stop-start)))
#sol_print(solved,matrix)

There are 24 valid hooks
Check correct grid: True


AttributeError: 'Matrix' object has no attribute 'final'

### Puzzle solution
<img src="https://www.janestreet.com/puzzles/wp-content/uploads/2018/03/20180228_hooks_3_ans.png" width="300" height="400">

In [158]:
correct = np.array([[9, 9, 9, 9, 9, 9, 9, 9, 9],
                    [8, 7, 7, 7, 7, 7, 7, 7, 9],
                    [8, 6, 6, 6, 6, 6, 6, 7, 9],
                    [8, 5, 4, 3, 2, 1, 6, 7, 9],
                    [8, 5, 4, 3, 2, 2, 6, 7, 9],
                    [8, 5, 4, 3, 3, 3, 6, 7, 9],
                    [8, 5, 4, 4, 4, 4, 6, 7, 9],
                    [8, 5, 5, 5, 5, 5, 6, 7, 9],                          
                    [8, 8, 8, 8, 8, 8, 8, 8, 9]],dtype=int)

solved=   np.array([[0, 0, 0, 0, 9, 9, 0, 9, 0],
                    [8, 7, 7, 0, 0, 7, 0, 7, 0],
                    [0, 0, 6, 6, 0, 6, 6, 7, 9],
                    [8, 0, 4, 0, 0, 1, 0, 0, 9],
                    [8, 5, 4, 3, 2, 2, 0, 0, 9],
                    [0, 5, 0, 3, 0, 3, 6, 7, 0],
                    [0, 0, 0, 4, 0, 4, 0, 7, 9],
                    [0, 5, 0, 5, 5, 0, 6, 0, 9],                          
                    [0, 8, 8, 8, 0, 0, 8, 8, 9]],dtype=int)

incorrect=np.array([[9, 9, 9, 9, 9, 9, 9, 9, 9],
                    [8, 7, 6, 6, 6, 6, 6, 6, 9],
                    [8, 7, 5, 5, 5, 5, 5, 6, 9],
                    [8, 7, 4, 3, 2, 1, 5, 6, 9],
                    [8, 7, 4, 3, 2, 2, 5, 6, 9],
                    [8, 7, 4, 3, 3, 3, 5, 6, 9],
                    [8, 7, 4, 4, 4, 4, 5, 6, 9],
                    [8, 7, 7, 7, 7, 7, 7, 7, 9],                          
                    [8, 8, 8, 8, 8, 8, 8, 8, 9]],dtype=int)

matrix = solved/correct

solution = np.ones((9,9))*-1


In [194]:
def forced(grid,hooks):
    grid[hooks==1] = 1
    changes = 1
    
    while changes >0:
        changes = 0
        for row,col in itertools.product(range(9),range(9)):
            if grid[row,col]==-1:
                # set up the numbers before and after the current grid spot 
                test =grid*hooks 
                above = test[:row,col]
                below = test[row+1:,col]
                left = test[row,:col]
                right  =test[row,col+1:]
            
                # work out the befores and afters for each rotation
                before = [above,below,left,right]   
                after = [below,above,right,left]
                avail_after = [[abs(i) for i in j] for j in after]
                param =[top_labels[col],bot_labels[col],left_labels[row],right_labels[row]]
                count_before = [np.sum(i > 0) for i in before ]
                num = hooks[row,col]

                #test each rotation
                for i in range(4):
                    if (param[i] != 0) & (np.sum(before[i] < 0) ==0):
                        if count_before[i] ==0:
                            if param[i] % num  != 0:
                                # if there is nothing before and the current cell does not divide the test number exactly set 0
                                grid[row,col]= 0
                                changes +=1
                            elif param[i] / hooks[row,col] not in avail_after[i]:
                                # if there is nothing before and test number/current number is not in the rest of the row set 0
                                grid[row,col]= 0
                                changes +=1
                            else:
                                
                                check_nums_still_avail = False                                
                                for n in range(len(avail_after[i])): 
                                    if avail_after[i][n] > 0:
                                        if param[i]/avail_after[i][n] in avail_after[i][n+1:]:
                                            check_nums_still_avail = True
                                            break
                                
                                if not check_nums_still_avail:
                                    #check if the numbers in the rest of the row can still make the test number if not set 1
                                    grid[row,col]= 1
                                    changes +=1
                                      
                                            
                        if count_before[i] ==1:
                            if num != param[i]/np.sum(before[i]):
                                #if a previous number is set and the current number is not the right number to complete the product set 0
                                grid[row,col] = 0
                                changes +=1
                            elif (num == param[i]/np.sum(before[i])) & (param[i]/np.sum(before[i]) not in avail_after[i]):
                                # if it is the right number to complete the product and that number is not in the rest of the row set 1
                                grid[row,col] = 1
                                changes +=1
                                
                test[row,col] = 0                        
                if np.max(label(test!=0,connectivity=1)) > 1:
                    #if placing zero would break connectivity set 1
                    grid[row,col]=1
                                                    
    return(grid)      
                    

In [199]:
solution = np.ones((9,9),dtype=int)*-1
start = time.perf_counter()
x = forced(solution,correct)
stop =  time.perf_counter()
print('Checking forced cells took {:0.4f} seconds'.format((stop-start)))
print(x)
np.sum(x==-1)

Checking forced cells took 0.1141 seconds
[[-1  0  0  0  1  1  0  1 -1]
 [ 1  1  1  0  0  1  0  1  0]
 [-1  0  1  1  0  1 -1  1  1]
 [ 1  0  1 -1 -1  1 -1 -1 -1]
 [ 1  1 -1  1 -1 -1 -1 -1 -1]
 [ 0  1  0  1  0 -1  1  1  0]
 [ 0  0 -1  1  0 -1 -1  1  1]
 [ 0 -1  0  1  1  0  1  0  1]
 [-1  1  1  1  0  0 -1  1 -1]]


23

In [200]:
#check if output matrix is correct
print(np.logical_or(x == matrix,x ==-1))

[[ True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True]]


In [210]:
def fill_rest(grid,hooks):
        for row,col in itertools.product(range(9),range(9)):
            if grid[row,col]==-1:
                for digit in range(2):
                    if possible(digit,row,col,grid,hooks):
                        grid[row,col] = digit
                        fill_rest(grid,hooks)
                        grid[row,col] = -1
                return
        print(grid*hooks,"\n",grid==matrix)
                                
                            
def possible(digit,row,col,grid,hooks):
    test = grid*hooks
    test[row,col] = hooks[row,col]*digit
    
    #check the placement doesn't break connectivity
    if np.max(label(test!=0,connectivity=1)) > 1:
        return False
    
    #check the placement doesn't exceed the permitted count of that number
    for num in range(2,10):
        if np.sum(test == num) > num:
               return False
        
    # final checks if the placement completes the grid    
    if np.sum(grid == -1) ==1:
        #check counts again
        for i in range(2,10):
            if np.sum(test == i) != i:
                return False
    
        #check rows and columns
        for i in range(9):
            row = test[i,:]
            col = test[:,i]
            if (col[col !=0][:2].prod() != top_labels[i]) & (top_labels[i] !=0):
                    return False
            if (col[col !=0][-2:].prod() != bot_labels[i]) & (bot_labels[i] !=0):
                    return False
            if (row[row !=0][:2].prod() != left_labels[i]) & (left_labels[i] !=0):
                    return False
            if (row[row !=0][-2:].prod() != right_labels[i]) & (right_labels[i] !=0):
                    return False
        
        
    return True
    

In [214]:
start = time.perf_counter()
fill_rest(x,correct)
stop =  time.perf_counter()
print('\n Filling in the rest took {:0.4f} seconds'.format((stop-start)))


[[0 0 0 0 9 9 0 9 0]
 [8 7 7 0 0 7 0 7 0]
 [0 0 6 6 0 6 6 7 9]
 [8 0 4 0 0 1 0 0 9]
 [8 5 4 3 2 2 0 0 9]
 [0 5 0 3 0 3 6 7 0]
 [0 0 0 4 0 4 0 7 9]
 [0 5 0 5 5 0 6 0 9]
 [0 8 8 8 0 0 8 8 9]] 
 [[ True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True]]

 Filling in the rest took 0.0031 seconds



<img src="https://www.janestreet.com/puzzles/wp-content/uploads/2018/03/20180228_hooks_3_ans.png" width="300" height="400">